In [28]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [30]:
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id": "productTitle"}).text.strip()

        title_value = title.text

        title_string = title_value.strip()
    except AttributeError:
        title_string = ""

    return title_string


def get_price(soup):
    try:
        price = soup.find("span", attrs={"class": "a-price aok-align-center reinventPricePriceToPayMargin priceToPay"}
                          ).find("span", attrs={"class": "a-offscreen"})
    except AttributeError:
        price = ""

    return price


def get_rating(soup):
    try:
        rating = soup.find("span", attrs={"class": "a-icon-alt"}).text.strip()
    except AttributeError:
        rating = ""

    return rating


def get_review_count(soup):
    try:
        review = soup.find("span", attrs={"id": "acrCustomerReviewText"}).text
    except:
        review = ""

    return review


def get_availablity(soup):
    try:
        availablity = soup.find(
            "div", attrs={"id": "availability"}).find("span", attrs={"class": "a-size-medium a-color-success"})
    except AttributeError:
        availablity = ""

    return availablity

In [31]:
if __name__ == "__main__":
    HEADERS = ({"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
                "Accept-Languages": "en-US, en;q = 0.5"})

    URL = "https://www.amazon.in/s?k=playstation+5&ref=nb_sb_ss_ts-doa-p_1_3"

    webpage = requests.get(URL, headers=HEADERS)

    soup = BeautifulSoup(webpage.content, "html.parser")

    links = soup.find_all("a", attrs={
        "class": "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})

    links_list = []

    for link in links:
        links_list.append(link.get("href"))

    data_dict = {"title": [], "price": [],
                 "rating": [], "reviews": [], "availablity": []}

    for link in links_list:
        new_webpage = requests.get("https://amazon.in"+link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        data_dict["title"].append(get_title(new_soup))
        data_dict["price"].append(get_price(new_soup))
        data_dict["rating"].append(get_rating(new_soup))
        data_dict["reviews"].append(get_review_count(new_soup))
        data_dict["availablity"].append(get_availablity(new_soup))

    amazon_df = pd.DataFrame.from_dict(data_dict)
    amazon_df['title'].replace("", np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_product_data.csv", header=True, index=False)